<a href="https://colab.research.google.com/github/TheMihirNaik/google-search-console-api-using-python/blob/main/GSC_API_25k%2B_Data%2C_Export_to_CSV%2C_Dimensions_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required python packages
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2

In [ ]:
# Import required packages
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2

# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "enter your own CLIENT_ID"
CLIENT_SECRET = "enter your own CLIENT_SECRET"
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print("Go to the following link in your browser: " + authorize_url)
auth_code = input("Enter your Authorization Code here:")
credentials = flow.step2_exchange(auth_code)
http = httplib2.Http()
creds = credentials.authorize(http)
webmasters_service = build('searchconsole', 'v1', http=creds)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=1085687348720-cpabg6ttpsqjji0j6pgbkb8sbcdcp4nt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code
Enter your Authorization Code here:4/1AWtgzh4uywHpd6wzFiPuORwCU30KMtMVC55ZeSbkpAV1LoNaTbhmOzrEyQM


# How to get a list of Sites we have in GSC Account?

In [ ]:
# Get a list of site in my Google Search Console Account
site_list = webmasters_service.sites().list().execute()

site_list

{'siteEntry': [{'siteUrl': 'https://moneymonkdigital.com/',
   'permissionLevel': 'siteOwner'},
  {'siteUrl': 'https://www.monocubed.com/', 'permissionLevel': 'siteFullUser'},
  {'siteUrl': 'https://www.tridentpbi.in/',
   'permissionLevel': 'siteFullUser'}]}

# How to get Google Search Console Analytics Data using API?

In [ ]:
# the website we want to get the data for
website = "https://www.monocubed.com/"

# build a request body
request_body = {
    "startDate"	: '2023-03-01',
    "endDate" : '2023-03-10',
    "dimensions" : ['QUERY', 'PAGE'],
    "rowLimit" : 25000,
    "dataState" : "final"
}

# get the response using request body
response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

In [ ]:
len(response_data['rows'])

25000

# How to use While Loop to get more than 25,000 Rows of Data

In [ ]:
# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2023-01-01',
      "endDate" : '2023-01-31',
      "dimensions" : ['QUERY', 'PAGE'],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 70826 rows of data


In [ ]:
len(all_responses)

70826

In [ ]:
all_responses[0]

{'keys': ['python backend framework',
  'https://www.monocubed.com/blog/top-python-frameworks/'],
 'clicks': 695,
 'impressions': 1743,
 'ctr': 0.3987378083763626,
 'position': 1.5243832472748136}

# Build a Dataframe using Pandas

In [ ]:
import pandas as pd

In [ ]:
#empty list to build a dataframe
data_for_df = []

for each in all_responses:

  temp = []

  #query
  temp.append(each['keys'][0])

  #page
  temp.append(each['keys'][1])

  #clicks
  temp.append(each['clicks'])

  #impressions
  temp.append(each['impressions'])

  #ctr
  temp.append(each['ctr'])

  #position
  temp.append(each['position'])

  data_for_df.append(temp)



In [ ]:
data_for_df[0]

['python backend framework',
 'https://www.monocubed.com/blog/top-python-frameworks/',
 695,
 1743,
 0.3987378083763626,
 1.5243832472748136]

In [ ]:
df = pd.DataFrame(data_for_df, columns=['query', 'page', 'clicks', 'impressions', 'ctr', 'position'])

In [ ]:
df

,query,page,clicks,impressions,ctr,position
0,python backend framework,https://www.monocubed.com/blog/top-python-fram...,695,1743,0.398738,1.524383
1,frontend frameworks,https://www.monocubed.com/blog/best-front-end-...,687,3789,0.181314,4.495909
2,front end frameworks,https://www.monocubed.com/blog/best-front-end-...,682,3724,0.183136,3.526047
3,python web framework,https://www.monocubed.com/blog/top-python-fram...,506,4810,0.105198,5.727235
4,frontend framework,https://www.monocubed.com/blog/best-front-end-...,395,1984,0.199093,4.552923
...,...,...,...,...,...,...
70821,장고 vs 노드,https://www.monocubed.com/blog/django-vs-node-...,0,1,0.000000,2.000000
70822,장고 vs 노드,https://www.monocubed.com/blog/django-vs-node-...,0,1,0.000000,2.000000
70823,코드이그나이터,https://www.monocubed.com/blog/best-ide-for-co...,0,1,0.000000,1.000000
70824,코드이그나이터 라라벨,https://www.monocubed.com/blog/codeigniter-vs-...,0,1,0.000000,20.000000


#Export to CSV or Excel

In [ ]:
df.to_excel('data.xlsx')

In [ ]:
df.to_csv('data.csv')

#How to use Dimensions & Dimension Groups?

In [ ]:
# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2023-01-01',
      "endDate" : '2023-01-02',
      "dimensions" : ['QUERY', 'PAGE'],
      "dimensionFilterGroups": [ { "filters": [
                  {"dimension": "QUERY", "expression": "python", "operator": "notContains" },
                  {"dimension": "PAGE", "expression": "python", "operator": "contains" },
                  {"dimension": "COUNTRY", "expression": "USA", "operator": "equals" }
                  ]}],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)

fetched up to 84 rows of data


In [ ]:
all_responses[0]

{'keys': ['apps developed',
  'https://www.monocubed.com/blog/apps-built-with-python/'],
 'clicks': 0,
 'impressions': 2,
 'ctr': 0,
 'position': 30}